In [202]:
import os
import sys
import time
import math
import keras
import datetime
import itertools
import numpy as np
import pandas as pd

from keras.layers import * 
from keras.models import Model
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Append TSPerf path to sys.path
nb_dir = os.path.split(os.getcwd())[0]
tsperf_dir = os.path.dirname(os.path.dirname(os.path.dirname(nb_dir)))
if tsperf_dir not in sys.path:
    sys.path.append(tsperf_dir)

from common.metrics import MAPE
import retail_sales.OrangeJuice_Pt_3Weeks_Weekly.common.benchmark_settings as bs

In [187]:
# Data paths
DATA_DIR = '../../data'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')

# Data parameters
MAX_STORE_ID = 137
MAX_BRAND_ID = 11

# Parameters of the model
PRED_HORIZON = 3
PRED_STEPS = 2
SEQ_LEN = 8
DYNAMIC_FEATURES = ['deal', 'feat']
STATIC_FEATURES = ['store', 'brand']

In [161]:
def df_from_cartesian_product(dict_in):
    """Generate a Pandas dataframe from Cartesian product of lists.
    
    Args: 
        dict_in (Dictionary): Dictionary containing multiple lists
        
    Returns:
        df (Dataframe): Dataframe corresponding to the Caresian product of the lists
    """
    from collections import OrderedDict
    from itertools import product
    od = OrderedDict(sorted(dict_in.items()))
    cart = list(product(*od.values()))
    df = pd.DataFrame(cart, columns=od.keys())
    return df

def gen_sequence(df, seq_len, seq_cols, start_timestep=0, end_timestep=None):
    """Reshape features into an array of dimension (time steps, features).  
    
    Args:
        df (Dataframe): Time series data of a specific (store, brand) combination
        seq_len (Integer): The number of previous time series values to use as input features
        seq_cols (List): A list of names of the feature columns 
        start_timestep (Integer): First time step you can use to create feature sequences
        end_timestep (Integer): Last time step you can use to create feature sequences
        
    Returns:
        A generator object for iterating all the feature sequences
    """
    data_array = df[seq_cols].values
    if end_timestep is None:
        end_timestep = df.shape[0]
    for start, stop in zip(range(start_timestep, end_timestep-seq_len+1), range(start_timestep+seq_len, end_timestep+1)):
        yield data_array[start:stop, :]


def gen_sequence_array(df_all, seq_len, seq_cols, start_timestep=0, end_timestep=None):
    """Combine feature sequences for all the combinations of (store, brand) into an 3d array.
    
    Args:
        df_all (Dataframe): Time series data of all stores and brands
        seq_len (Integer): The number of previous time series values to use as input features
        seq_cols (List): A list of names of the feature columns 
        start_timestep (Integer): First time step you can use to create feature sequences
        end_timestep (Integer): Last time step you can use to create feature sequences
        
    Returns:
        seq_array (Numpy Array): An array of the feature sequences of all stores and brands    
    """
    seq_gen = (list(gen_sequence(df_all[(df_all['store']==cur_store) & (df_all['brand']==cur_brand)], \
                                 seq_len, seq_cols, start_timestep, end_timestep)) \
              for cur_store, cur_brand in itertools.product(df_all['store'].unique(), df_all['brand'].unique()))
    seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
    return seq_array

def static_feature_array(df_all, total_timesteps, seq_cols):
    """Generate an arary which encodes all the static features.
    """
    fea_df = data_filled.groupby(['store', 'brand']). \
                         apply(lambda x: x.iloc[:total_timesteps,:]). \
                         reset_index(drop=True)
    fea_array = fea_df[seq_cols].values
    return fea_array

In [5]:
r = 0
print('---- Round ' + str(r+1) + ' ----')
train_df = pd.read_csv(os.path.join(TRAIN_DIR, 'train_round_'+str(r+1)+'.csv'))
train_df['move'] = train_df['logmove'].apply(lambda x: round(math.exp(x)))
train_df.drop('logmove', axis=1, inplace=True)
print(train_df.head(3))
print('')
# Fill missing values
store_list = train_df['store'].unique()
brand_list = train_df['brand'].unique()
week_list = range(bs.TRAIN_START_WEEK, bs.TEST_END_WEEK_LIST[r]+1)
d = {'store': store_list,
     'brand': brand_list,
     'week': week_list}        
data_grid = df_from_cartesian_product(d)
data_filled = pd.merge(data_grid, train_df, how='left', 
                        on=['store', 'brand', 'week'])
print('Number of missing rows is {}'.format(data_filled[data_filled.isnull().any(axis=1)].shape[0]))
print('')
data_filled = data_filled.groupby(['store', 'brand']). \
                          apply(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
print(data_filled.head(3))
print('')

---- Round 1 ----
   store  brand  week  constant    price1    price2    price3    price4  \
0      2      1    40         1  0.060469  0.060497  0.042031  0.029531   
1      2      1    46         1  0.060469  0.060312  0.045156  0.046719   
2      2      1    47         1  0.060469  0.060312  0.045156  0.046719   

     price5    price6    price7    price8    price9   price10   price11  deal  \
0  0.049531  0.053021  0.038906  0.041406  0.028906  0.024844  0.038984     1   
1  0.049531  0.047813  0.045781  0.027969  0.042969  0.042031  0.038984     0   
2  0.037344  0.053021  0.045781  0.041406  0.048125  0.032656  0.038984     0   

   feat     profit  move  
0   0.0  37.992326  8256  
1   0.0  30.126667  6144  
2   0.0  30.000000  3840  

Number of missing rows is 6204

   brand  store  week  constant    price1    price2    price3    price4  \
0      1      2    40       1.0  0.060469  0.060497  0.042031  0.029531   
1      1      2    41       1.0  0.060469  0.060497  0.042031  0.

In [127]:
# Normalize the dataframe of features
cols_normalize = data_filled.columns.difference(['store','brand','week'])
min_max_scaler = MinMaxScaler()
data_filled_scaled = pd.DataFrame(min_max_scaler.fit_transform(data_filled[cols_normalize]), 
                                  columns=cols_normalize, 
                                  index=data_filled.index)
data_filled_scaled.head()

,constant,deal,feat,move,price1,price10,price11,price2,price3,price4,price5,price6,price7,price8,price9,profit
0,0.0,1.0,0.0,0.011436,1.0,0.493088,1.0,0.993939,0.686433,0.434783,1.0,1.0,0.731481,0.977528,0.485356,0.38567
1,0.0,1.0,0.0,0.011436,1.0,0.493088,1.0,0.993939,0.686433,0.434783,1.0,1.0,0.731481,0.977528,0.485356,0.38567
2,0.0,1.0,0.0,0.011436,1.0,0.493088,1.0,0.993939,0.686433,0.434783,1.0,1.0,0.731481,0.977528,0.485356,0.38567
3,0.0,1.0,0.0,0.011436,1.0,0.493088,1.0,0.993939,0.686433,0.434783,1.0,1.0,0.731481,0.977528,0.485356,0.38567
4,0.0,1.0,0.0,0.011436,1.0,0.493088,1.0,0.993939,0.686433,0.434783,1.0,1.0,0.731481,0.977528,0.485356,0.38567


In [68]:
data_sub = data_filled[(data_filled.brand==1) & (data_filled.store==2)]
data_sub.shape

(99, 19)

In [120]:
start_timestep = 0
end_timestep = bs.TRAIN_END_WEEK_LIST[r]-bs.TRAIN_START_WEEK-PRED_HORIZON+1

train_input1 = gen_sequence_array(data_filled, SEQ_LEN, ['move'], start_timestep, end_timestep)

train_input1.shape

(78518, 8, 1)

In [121]:
start_timestep = PRED_HORIZON
end_timestep = bs.TRAIN_END_WEEK_LIST[r]-bs.TRAIN_START_WEEK+1

train_input2 = gen_sequence_array(data_filled, SEQ_LEN, DYNAMIC_FEATURES, start_timestep, end_timestep)

train_input2.shape

(78518, 8, 2)

In [105]:
train_input = np.concatenate((train_input1, train_input2), axis=2)
train_input.shape

(78518, 8, 3)

In [125]:
#train_output
start_timestep = SEQ_LEN+1
end_timestep = bs.TRAIN_END_WEEK_LIST[r]-bs.TRAIN_START_WEEK+1
train_output = gen_sequence_array(data_filled, PRED_STEPS, ['move'], start_timestep, end_timestep)
train_output = np.squeeze(train_output)
train_output.shape

(78518, 2)

(78518, 2)

In [43]:
store_col = np.full([train_input1.shape[0],1], 1)
brand_col = np.full([train_input1.shape[0],1], 2)
np.concatenate((store_col, brand_col), axis=1)

(88, 2)

In [49]:
bs.TRAIN_END_WEEK_LIST[r]-bs.TRAIN_START_WEEK-sequence_len+1

88

In [14]:
# Test MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data_sub)
data_sub_scaled = scaler.transform(data_sub)
data_sub_scaled

array([[0.        , 0.        , 0.        , ..., 0.        , 0.73084502,
        0.11508554],
       [0.        , 0.        , 0.01020408, ..., 0.        , 0.73084502,
        0.11508554],
       [0.        , 0.        , 0.02040816, ..., 0.        , 0.73084502,
        0.11508554],
       ...,
       [0.        , 0.        , 0.97959184, ..., 0.        , 0.3278259 ,
        0.21617418],
       [0.        , 0.        , 0.98979592, ..., 0.        , 0.3278259 ,
        0.21617418],
       [0.        , 0.        , 1.        , ..., 0.        , 0.3278259 ,
        0.21617418]])

In [17]:
data_sub_scaled[0,]

array([0.        , 0.        , 0.        , 0.        , 1.        ,
       1.        , 0.62921348, 0.25477707, 1.        , 1.        ,
       0.70103093, 0.97435897, 0.41148325, 0.35294118, 1.        ,
       1.        , 0.        , 0.73084502, 0.11508554])

In [12]:
sa = gen_sequence_array(data_sub, sequence_length, sequence_cols)
sa.shape

(91, 8, 2)

In [178]:
total_timesteps = bs.TRAIN_END_WEEK_LIST[r]-bs.TRAIN_START_WEEK-SEQ_LEN-PRED_HORIZON+2
train_input2 = static_feature_array(data_filled, total_timesteps, ['store', 'brand'])

In [179]:
train_input2.shape

(78518, 2)

In [191]:
# Model definition
def create_dcnn_model(seq_len, kernel_size=2, n_filters=4, n_input_series=1, n_outputs=1):
    # Sequential input
    seq_in = Input(shape=(seq_len, n_input_series), name='input_1')
    # Categorical input
    cat_fea_in = Input(shape=(2,), dtype='uint8', name='input_2')
    store_id = Lambda(lambda x: x[:, 0, None])(cat_fea_in)
    brand_id = Lambda(lambda x: x[:, 1, None])(cat_fea_in)
    store_embed = Embedding(MAX_STORE_ID+1, 7, input_length=1)(store_id)
    brand_embed = Embedding(MAX_BRAND_ID+1, 4, input_length=1)(brand_id)
    
    # Dilated convolutional layers
    c1 = Conv1D(filters=n_filters, kernel_size=kernel_size, dilation_rate=1, 
                padding='causal', activation='relu', name='conv1d_1')(seq_in)
    c2 = Conv1D(filters=n_filters, kernel_size=kernel_size, dilation_rate=2, 
                padding='causal', activation='relu', name='conv1d_2')(c1)
    c3 = Conv1D(filters=n_filters, kernel_size=kernel_size, dilation_rate=4, 
                padding='causal', activation='relu', name='conv1d_3')(c2)
    # Skip connections
    c4 = concatenate([c1, c3])
    # Output of convolutional layers 
    conv_out = Conv1D(8, 1, activation='relu')(c4)
    conv_out = Dropout(0.25)(conv_out)
    conv_out = Flatten()(conv_out)
    
    # Concatenate with categorical features
    x = concatenate([conv_out, Flatten()(store_embed), Flatten()(brand_embed)])
    #x = BatchNormalization()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(n_outputs, activation='linear')(x)
    
    model = Model(inputs=[seq_in, cat_fea_in], outputs=output)
    adam = optimizers.Adam(lr=0.01)
    model.compile(loss='mse', optimizer=adam, metrics=['mae'])
    return model

In [192]:
model = create_dcnn_model(seq_len=SEQ_LEN, n_outputs=PRED_STEPS)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 8, 1)         0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 8, 4)         12          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 8, 4)         36          conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 8, 4)         36          conv1d_2[0][0]                   
__________________________________________________________________________________________________
concatenat

In [193]:
model.fit([train_input1, train_input2], train_output, nb_epoch=25, batch_size=2, validation_data=([train_input1, train_input2], train_output))

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Train on 78518 samples, validate on 78518 samples
Epoch 1/25
78518/78518 [==============================] - 133s 2ms/step - loss: 386293512.0778 - mean_absolute_error: 8087.8466 - val_loss: 374295882.4036 - val_mean_absolute_error: 7858.6402
Epoch 2/25
78518/78518 [==============================] - 132s 2ms/step - loss: 378488205.3290 - mean_absolute_error: 7961.6788 - val_loss: 373397352.7377 - val_mean_absolute_error: 7849.8510
Epoch 3/25
78518/78518 [==============================] - 133s 2ms/step - loss: 377414270.3836 - mean_absolute_error: 7909.0688 - val_loss: 376472132.8982 - val_mean_absolute_error: 7422.8482
Epoch 4/25
78518/78518 [==============================] - 132s 2ms/step - loss: 376378657.4870 - mean_absolute_error: 7900.8502 - val_loss: 378515494.8343 - val_mean_absolute_error: 7220.1104
Epoch 5/25
78518/78518 [==============================] - 132s 2ms/step - loss: 375873679.0266 - mean_absolute_error: 7861.7805 - val_loss: 386010663.2796 - val_mean_absolute_error: 

In [197]:
np.mean(train_output)

9905.389

In [203]:
from keras.utils import multi_gpu_model

try:
    model = multi_gpu_model(model, cpu_merge=False)
    print("Training using multiple GPUs..")
except:
    print("Training using single GPU or CPU..")
start_time = time.time()
model.fit([train_input1, train_input2], train_output, epochs=5, batch_size=2, validation_data=([train_input1, train_input2], train_output))
print(time.time()-start_time)

Training using single GPU or CPU..
Train on 78518 samples, validate on 78518 samples
Epoch 1/5
78518/78518 [==============================] - 129s 2ms/step - loss: 371783279.8227 - mean_absolute_error: 7720.2254 - val_loss: 375786319.9688 - val_mean_absolute_error: 6926.7757
Epoch 2/5
78518/78518 [==============================] - 130s 2ms/step - loss: 371858775.8101 - mean_absolute_error: 7743.6835 - val_loss: 374005735.5173 - val_mean_absolute_error: 7079.4063
Epoch 3/5
78518/78518 [==============================] - 130s 2ms/step - loss: 371711579.5102 - mean_absolute_error: 7737.9117 - val_loss: 374819726.8688 - val_mean_absolute_error: 6959.9123
Epoch 4/5
78518/78518 [==============================] - 130s 2ms/step - loss: 372115131.8486 - mean_absolute_error: 7743.9186 - val_loss: 372810391.6923 - val_mean_absolute_error: 7005.5653
Epoch 5/5
78518/78518 [==============================] - 130s 2ms/step - loss: 372776029.6781 - mean_absolute_error: 7742.7033 - val_loss: 374814529.70

In [223]:
import tensorflow as tf
tf.test.is_gpu_available()

False